In [ ]:
def most_common_distribution( nouns_list, find_all_occ ):
    
    
    dcc = find_all_occ( nouns_list )
    dcc['PoS'] = dcc['Caption'].apply(nltk.word_tokenize)
    dcc['PoS'] = dcc['PoS'].apply(nltk.pos_tag)
    
    
    
    CC_cnt = {}
    
    def find_CC( sl ):
        
        for pair in sl:
            
            word = pair[0]
            tag = pair[1]
            
            if not (tag in CC_cnt.keys()):
                CC_cnt[tag] = {}
            
            if not (word in CC_cnt[ tag ].keys()):
                CC_cnt[tag][word] = 0
                
            CC_cnt[tag][word] += 1
    
    
    dcc['PoS'].apply( find_CC )
    
    
    
    # Crunch

    if 'VBN' in CC_cnt.keys():
        
        CC_cnt['JJ'].pop('white', None)

        # table table
        for noun in nouns_list:
            CC_cnt['JJ'].pop(noun, None)
    
    if 'VBN' in CC_cnt.keys():
        CC_cnt['VBN'].pop('isolated', None)
    
    
    CC_mc = {}
    
    for tag in CC_cnt.keys():
        dic = CC_cnt[tag]
        if len(dic) > 0:
            CC_mc[tag] = max(dic, key=dic.get)
    
    
    
    
    
    return CC_mc

In [ ]:
def build_multiroot( nouns, find_all_occ, clusters ):
    

    
    CC_mcs = {
        n: most_common_distribution( [n], find_all_occ )
        for n in nouns
    }
    
    
    intro = 'The image of'
    
    
    markup = [intro]
    
    for noun in nouns:
    
#         dt = ''
#         if 'DT' in CC_mcs[ nouns[0] ].keys():
#             dt = CC_mcs[ nouns[0] ][ 'DT' ]
            
        jj = ''
        if 'JJ' in CC_mcs[ noun ].keys():
            jj = CC_mcs[ noun ][ 'JJ' ]
    
    
        noun_c = clusters[noun]

        noun_c = noun_c.split(' ')
        plural = noun_c[-1] 
        noun_c = noun_c[:-1]
        if jj != '':
            noun_c.append(jj)
        noun_c.append( plural )
        
        markup.append(' '.join(noun_c))
        markup.append('and')

    
    sentence = ' '.join(markup[:-1])
    
    return sentence + '. '
    
    
    

In [ ]:
def parse_siblings( nouns, clusters ):
    
    length = len(nouns)
    
    cum = f'{clusters[nouns[0]]}'
    
    for i in range(1, length-1):
        
        cum += f', {clusters[nouns[i]]}'
        
    if length != 1:
        cum += f' and {clusters[nouns[-1]]}'
    
    
    return cum
    

In [ ]:
def build_root( root_nouns, root_children, find_all_occ, clusters ):
    
    
    intro = 'The image of'
    sentence = ''
    
    if len( root_nouns ) == 1:
        
#         print(root_children)
        ch = parse_siblings( root_children, clusters )
        sentence = f'{intro} {clusters[ root_nouns[0] ]} with {ch}. '
        
        return 2, sentence
    
    else:
        
        sentence = build_multiroot( root_nouns, find_all_occ, clusters )
        
        return 1, sentence
        
    
    
    
    
    

In [ ]:
def choose_parent( noun, parents, fao ):
    
    numbers_of_sent = {}
    
    for parent in parents:
        
        # родители соревнуются в том, какой из них уделял больше внимания
        numbers_of_sent[ parent ] = len(fao([ parent, noun ]))
        
    winner = max( numbers_of_sent, key=numbers_of_sent.get )
    return winner
    
    

In [1]:
def build_follow_up( nouns, parents, fao, clusters ):
    
    sent = ''
    
    families = {
        p: []
        for p in parents
    }
    
    for noun in nouns:
        
        parent = choose_parent( noun, parents, fao )
#         print('parent of', noun, 'is', parent)
        
        families[ parent ].append( noun )
        
    for parent, kids_list in families.items():
        
        for kid in kids_list:
            
            mutual_MC = most_common_distribution( [parent, kid], fao )
            
            personal_MC = most_common_distribution( [kid], fao )
            
            
        
            
            
            mc_jj = personal_MC['JJ'] if ( 'JJ' in personal_MC.keys() ) else ''
            mc_vbz = mutual_MC['VBZ'] if ( 'VBZ' in mutual_MC.keys() ) else ''
            mc_vbn = mutual_MC['VBN'] if ( 'VBN' in mutual_MC.keys() ) else ''
            mc_in = mutual_MC['IN'] if ( 'IN' in mutual_MC.keys() ) else ''
            
            kid_c = clusters[kid]
            kid_c = kid_c.split(' ')
            plural = kid_c[-1] 
            kid_c = kid_c[:-1]
            if mc_jj != '':
                kid_c.append(mc_jj)
            kid_c.append( plural )
            kid_c = ' '.join(kid_c)
            
            sub_sent = ' '.join([
                
                kid_c,
                mc_vbz,
                mc_vbn,
                mc_in,
                'the',
                parent
                
            ]) + '.'
            
#             print(sub_sent)
            sent += ' ' + sub_sent.capitalize()
            
        
#         return mutual_MC
    
    
    return sent

In [ ]:
def stitch( tree, fao, clusters ):
    
    
    steps = list(tree.keys())
    
    starter, caption = build_root( tree[steps[0]], tree[steps[1]], fao, clusters )
    

    for i in range(starter, len(steps)):
        
        caption += build_follow_up( tree[steps[i]], tree[steps[i-1]], fao, clusters )
        
        
        
    return caption

In [ ]:
print('''
Imported functions for use outside:

\tstitch( tree, fao, clusters )

''')